# Predicting Latent Heating by Adding Changes in Features

Mainly looking at how prediction would work if the difference in subsequent
data points was added as a feature itself. There will be some feature selection
and data analysis to determine good features to use and what data points should
be considered extreme LH.

In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

# Datafile
df = pd.read_csv("./data_table_NA850_nonfiltered_updated.csv")

In [2]:
df.head()

,AirTemp,QV,Omega,SeaLevPress,UWinds,VWinds,LH,Time,Lat,Lon
0,7.060846,7.267001,0.000642,1021.565625,-11.894431,-5.276164,12.888080,2000010100,25.064459,-35.368896
1,7.100183,7.474899,-0.000296,1021.084141,-11.230261,-3.172798,-10.217120,2000010103,24.480576,-36.771637
2,7.068231,7.335827,0.000323,1020.302969,-10.246526,-1.993664,0.511768,2000010106,24.649042,-37.882874
3,7.328638,7.064822,0.000828,1020.585156,-9.889462,-1.943067,-16.420087,2000010109,24.122194,-38.753021
4,7.598169,7.333828,0.000373,1022.448281,-10.461138,-1.204948,-6.642615,2000010112,24.368916,-40.022919


In [3]:
# Add some diff data and drop Lat/Lon/Time
df['dAirTemp'] = df.AirTemp.diff()
df['dQV'] = df.QV.diff()
df['dOmega'] = df.Omega.diff()
df['dSeaLevPress'] = df.SeaLevPress.diff()
df['dUWinds'] = df.UWinds.diff()
df['dVWinds'] = df.VWinds.diff()
df['dLat'] = df.Lat.diff()
df['dLon'] = df.Lon.diff()

df.loc[0, 'dLat'] = 0
df.loc[0, 'dLon'] = 0
df.loc[0, 'dUWinds'] = 0
df.loc[0, 'dVWinds'] = 0
df.loc[0, 'dSeaLevPress'] = 0
df.loc[0, 'dOmega'] = 0
df.loc[0, 'dAirTemp'] = 0
df.loc[0, 'dQV'] = 0

df['Time'] = pd.to_datetime(df.Time, format="%Y%m%d%H")
df = df.set_index('Time')
df = df.drop(axis=1, labels=["Lat", "Lon"])

In [4]:
# Scale the data using a RobustScaler
scaler = RobustScaler()
df = pd.DataFrame(scaler.fit_transform(df),
                  columns=df.columns,
                  index=df.index)

## Quick Look at Kendall Correlations

In [5]:
df.describe()

,AirTemp,QV,Omega,SeaLevPress,UWinds,VWinds,LH,dAirTemp,dQV,dOmega,dSeaLevPress,dUWinds,dVWinds,dLat,dLon
count,1.732000e+03,1.732000e+03,1.732000e+03,1.732000e+03,1732.000000,1732.000000,1.732000e+03,1732.000000,1732.000000,1732.000000,1732.000000,1732.000000,1732.000000,1732.000000,1732.000000
mean,-9.374876e-02,-2.657104e-01,-3.329808e-01,1.459076e-01,0.012800,-0.028583,6.680829e-01,0.048794,0.005509,0.001068,-0.032544,0.021760,0.002498,0.032905,1.077419
std,1.788178e+00,7.234944e-01,1.504368e+00,8.840828e-01,0.682365,0.844551,1.952211e+00,1.849300,1.490525,1.473869,0.692870,1.160012,1.268670,3.540790,8.526088
min,-6.304448e+00,-2.204174e+00,-1.577670e+01,-1.982083e+00,-2.421385,-2.644464,-5.868978e+00,-11.342577,-11.828216,-10.692983,-6.221531,-8.564673,-9.083783,-36.277584,-54.418667
25%,-5.653561e-01,-7.241540e-01,-5.758553e-01,-4.368651e-01,-0.500500,-0.549006,-2.073239e-01,-0.506030,-0.484391,-0.497573,-0.532421,-0.490370,-0.497380,-0.488911,-0.504643
50%,-1.064524e-15,-1.804655e-16,-1.923273e-17,-1.632657e-14,0.000000,0.000000,1.962406e-17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.346439e-01,2.758460e-01,4.241447e-01,5.631349e-01,0.499500,0.450994,7.926761e-01,0.493970,0.515609,0.502427,0.467579,0.509630,0.502620,0.511089,0.495357
max,7.090267e+00,8.210361e-01,4.854562e+00,3.974510e+00,2.267694,4.169482,2.432005e+01,20.371825,11.220097,9.983576,5.714968,7.818008,8.792477,33.672500,83.203138


<AxesSubplot:>

In [6]:
fig1, ax1 = plt.subplots(figsize=(12, 9))
sns.heatmap(abs(df.corr('kendall')), ax=ax1, annot=True)

The correlations here are overall fairly weak. Because of the lack of options
I'll go ahead and sort the correlations on Latent Heating from highest to
lowest correlation. I'll then create several predictors where a feature is
added in descending order of correlation.


In [9]:
desc_corr = abs(df.corr('kendall'))[['LH']].sort_values(by='LH', ascending=False)
desc_corr

,LH
LH,1.000000
Omega,0.455173
QV,0.406391
dQV,0.142112
SeaLevPress,0.140413
dOmega,0.130199
AirTemp,0.121293
dLat,0.095257
dLon,0.061879
UWinds,0.041764


## Linear Regression

In [16]:
data_Y = df['LH']
data_X = df[desc_corr.index[1:]]